In [ ]:
!pip install pdf2image

In [ ]:
!conda install -c conda-forge poppler

In [ ]:
import numpy as np
import json
import glob

# Extract image
import pytesseract
import cv2
from PIL import Image

# Gensim
import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel

# spacy
import spacy
from nltk.corpus import stopwords

# lda visual
import pyLDAvis
import pyLDAvis.gensim

# PDF 
from pdf2image import convert_from_path

import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)

# Step 1, OCR
## Extrak Text From PDF File

In [ ]:
# Path ke file PDF
pdf_file = "assets/jurnal.pdf"
index = 0
result = []

# Fungsi untuk mengekstrak teks dari file PDF dan menampilkan contour
def extract_text_and_show_contours(pdf_path):
    # Ubah file PDF menjadi gambar (gambar per halaman)
    images = []
    
    try:
        images = convert_pdf_to_images(pdf_path)
    except Exception as e:
        print(f"Error: {e}")

    # Loop melalui setiap gambar (halaman) dan ekstrak teks serta tampilkan contour
    for i, image in enumerate(images):
        text = show_text_contours(image)
        print(f"Hasil ekstraksi teks dari halaman {i + 1}:\n{text}\n")
#         show_text_contours(image)

# Fungsi untuk mengubah file PDF menjadi daftar gambar
def convert_pdf_to_images(pdf_path):
    images = []

    # Fungsi untuk mengonversi PDF menjadi gambar
    pdf_images = convert_from_path(pdf_path)

    # Simpan setiap gambar sebagai file gambar terpisah
    for i, image in enumerate(pdf_images):
        images.append(f"convert_to_img/page_{i + 1}.png")
        image.save(f'convert_to_img/page_{i + 1}.png', 'PNG')
                
    return images

# Fungsi untuk mengekstrak teks dari gambar menggunakan pytesseract
def extract_text_from_image(image):

    ocr_result = pytesseract.image_to_string(image)
    ocr_result = ocr_result.split("\n")

    for line in ocr_result:

        result.append(line)
                
    return ocr_result

# Fungsi untuk menampilkan contour dari gambar
def show_text_contours(image):
    image = cv2.imread(image)
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    blur = cv2.GaussianBlur(gray, (7,7), 0)
    thresh = cv2.threshold(blur, 0, 255, cv2.THRESH_BINARY_INV + cv2.THRESH_OTSU)[1]
    kernal = cv2.getStructuringElement(cv2.MORPH_RECT, (3, 13))
    dilate = cv2.dilate(thresh, kernal, iterations=1)
    
    cnts = cv2.findContours(dilate, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    cnts = cnts[0] if len(cnts) == 2 else cents[1]
    cnts = sorted(cnts, key=lambda x: cv2.boundingRect(x)[0])
    
    for c in cnts:
        x, y, w, h = cv2.boundingRect(c)

        # if statement berikut untuk mengambil bagian isi dari image, jika tidak maka semua kata akan di contours

        if h > 200 and w > 20:
            roi = image[y:y+h, x:x+h]
            cv2.imwrite(f"temp/jurnal_countor_{index}.png", roi)
            cv2.rectangle(image, (x, y), (x+w, y+h), (36, 255, 12), 2)
            
            extract_text_from_image(roi)
                
            index += 1


In [ ]:
extract_text_and_show_contours(pdf_file)

In [ ]:
result